# Import Libries

In [3]:
# !pip install emoji

In [4]:
# !python -m spacy download en_core_web_md

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import emoji
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import spacy


## Load X_train,X_eval,y_train,y_eval

In [6]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_eval = pd.read_csv('X_eval.csv')
y_eval = pd.read_csv('y_eval.csv')

In [7]:
X_train

,text
0,room belongs year renovated shared apartment w...
1,welcome join u experience wonder unique world ...
2,apartment lisbon bedroom capacity people apart...
3,apartamento super acolhedor bem mobilado equip...
4,looking one characteristic neighbourhood city ...
...,...
9991,b license number b alo nome isabel respons vel...
9992,lovely newly decorated private bedroom located...
9993,simple small cozy bedroom one best location li...
9994,luxury apartment belem dazzling rooftop view t...


In [8]:
X_train = X_train['text']

print(type(X_train))

<class 'pandas.core.series.Series'>


In [9]:
X_eval = X_eval['text']

print(type(X_eval))

<class 'pandas.core.series.Series'>


In [10]:
y_train

,unlisted
0,0
1,0
2,0
3,0
4,1
...,...
9991,0
9992,1
9993,0
9994,1


In [11]:
y_train = y_train['unlisted']

print(type(y_train))

<class 'pandas.core.series.Series'>


In [12]:
y_eval = y_eval['unlisted']

print(type(y_eval))

<class 'pandas.core.series.Series'>


In [13]:
X_train

0       room belongs year renovated shared apartment w...
1       welcome join u experience wonder unique world ...
2       apartment lisbon bedroom capacity people apart...
3       apartamento super acolhedor bem mobilado equip...
4       looking one characteristic neighbourhood city ...
                              ...                        
9991    b license number b alo nome isabel respons vel...
9992    lovely newly decorated private bedroom located...
9993    simple small cozy bedroom one best location li...
9994    luxury apartment belem dazzling rooftop view t...
9995    new apartment nearly year building elevator fi...
Name: text, Length: 9996, dtype: object

In [14]:
y_train

0       0
1       0
2       0
3       0
4       1
       ..
9991    0
9992    1
9993    0
9994    1
9995    0
Name: unlisted, Length: 9996, dtype: int64

In [15]:
X_eval

0       typical area lisbon bica located mn metro baix...
1       located quiet peaceful place although close em...
2       ight apartment guest near marqu pombal lisbon ...
3       situated highest seven hill beautiful apartmen...
4       cozy apartment first floor heart alfama locate...
                              ...                        
2495    situated ericeira bedroom apartment sleep peop...
2496    b license number b exemptsou maria vivo sempre...
2497    bem vindos melhor forma conhecer portugal km c...
2498    quarto pequeno cora lisboa balc muita luz metr...
2499    casa rec renovada localizada edif cio hist ric...
Name: text, Length: 2500, dtype: object

In [16]:
y_eval

0       0
1       0
2       0
3       0
4       0
       ..
2495    0
2496    0
2497    0
2498    0
2499    0
Name: unlisted, Length: 2500, dtype: int64

## Feature Engineering

In [17]:
# Load the pre-trained GloVe word vectors
nlp = spacy.load('en_core_web_md')

In [18]:
# Convert text to GloVe vectors
def get_glove_vectors(text):
    doc = nlp(text)
    return doc.vector
X_train_glove = np.array([get_glove_vectors(text) for text in X_train])
X_eval_glove = np.array([get_glove_vectors(text) for text in X_eval])

In [19]:
# Create a TF-IDF vectorizer to convert text into numerical features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_eval_tfidf = vectorizer.transform(X_eval)

In [20]:
# Combine GloVe and TF-IDF features
X_train_combined = np.concatenate((X_train_glove, X_train_tfidf.toarray()), axis=1)
X_eval_combined = np.concatenate((X_eval_glove, X_eval_tfidf.toarray()), axis=1)

In [21]:
print(type(X_train_combined))

<class 'numpy.ndarray'>


# Classification Models

### KNN

In [22]:
# Train KNN classifier
knn = KNeighborsClassifier()
knn.fit(X_train_combined, y_train)

KNeighborsClassifier()

### Logistic regression

In [24]:
# Train logistic regression classifier
lr = LogisticRegression()
lr.fit(X_train_combined, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### MLP

In [25]:
# Train MLP classifier
mlp = MLPClassifier()
mlp.fit(X_train_combined, y_train)

MLPClassifier()

# EVALUATION AND RESULTS

In [26]:
# Evaluate the models
models = {
    'Logistic Regression': lr,
    'KNN': knn,
    'MLP': mlp
}

evaluation_results = {}
for model_name, model in models.items():
    y_pred = model.predict(X_eval_combined)
    accuracy = accuracy_score(y_eval, y_pred)
    precision = precision_score(y_eval, y_pred)
    recall = recall_score(y_eval, y_pred)
    f1 = f1_score(y_eval, y_pred)

    evaluation_results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1
    }

In [27]:
# Find the best model based on F1-score
best_model = max(evaluation_results, key=lambda x: evaluation_results[x]['F1-score'])

In [28]:
best_model

'MLP'

In [29]:
merged_df_test = pd.read_csv('merged_df_test.csv')
merged_df_test

,text
0,<b>The space</b><br />Apartment located in the...
1,"IMPORTANT: In response to COVID-19, this prope..."
2,"Bright, beautiful, and spacious. This four-bed..."
3,Charming Apartment Close to the Bay of Cascais...
4,"Se procura umas férias perto da praia, casino ..."
...,...
1384,O melhor ponto de partida para descobrir Lisbo...
1385,Casa de Baixo is a cozy and comfortable apartm...
1386,Beautiful and peaceful apartment with a superb...
1387,The apartment in Lisbon has capacity for 2 pe...


In [30]:
# Make predictions on the test dataset using the best model
X_test = merged_df_test['text']
X_test_glove = np.array([get_glove_vectors(text) for text in X_test])
X_test_tfidf = vectorizer.transform(X_test)
X_test_combined = np.concatenate((X_test_glove, X_test_tfidf.toarray()), axis=1)
predictions = models[best_model].predict(X_test_combined)

In [32]:
models = {
    'Logistic Regression': lr,
    'KNN': knn,
    'MLP': mlp
}

for model_name, model in models.items():
    y_pred = model.predict(X_eval_combined)
    accuracy = accuracy_score(y_eval, y_pred)
    precision = precision_score(y_eval, y_pred, average='macro')
    recall = recall_score(y_eval, y_pred, average='macro')
    f1 = f1_score(y_eval, y_pred, average='macro')

    print(f"Evaluation results for {model_name}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print("-----------------------------")

Evaluation results for Logistic Regression:
Accuracy: 0.7032
Precision: 0.4882640972990049
Recall: 0.49853967714875796
F1-score: 0.43407577710090595
-----------------------------
Evaluation results for KNN:
Accuracy: 0.6584
Precision: 0.5012512495260418
Recall: 0.5007134551749675
F1-score: 0.4837862957471728
-----------------------------
Evaluation results for MLP:
Accuracy: 0.6244
Precision: 0.4808922676930527
Recall: 0.48585589856436306
F1-score: 0.4770632943876361
-----------------------------


In [33]:
# Create a DataFrame with the predictions
predictions_df = pd.DataFrame({'prediction': predictions})

# Generate indices using range() function
predictions_df['index'] = range(len(predictions_df))

In [ ]:

# Save the DataFrame to a CSV file
predictions_df.to_csv('Predictions_26.csv', index=False)